# Исследование надежности заемщиков.

<p>Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.</p>
<p>Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.</p>

**Цель исследования**
<p>Заказчиком поставлены следующие вопросы:</p>

* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?


## Обзор данных

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Данные представлены 12 столбцами, в столбцах `days_employed` и `total_income` некоторые значения пропущены. Числовые данные представлены в 7 столбцах, из них 5 имеют количественные значения, а 2 являются индентификаторами для строковых столбцов `education` и `family_status`.

## Обработка данных

### Заполнение пропусков

Пропуски наблюдаются в столбцах `days_employed` и `total_income`. Доля пропущенных значений составляет:

In [2]:
days_employed_missed = 1 - data['days_employed'].count()/len(data)
print("Доля пропущенных значений в столбце 'days_employed': {0:.1%}".format(days_employed_missed))

total_income_missed = 1 - data['total_income'].count()/len(data)
print("Доля пропущенных значений в столбце 'total_income': {0:.1%}".format(total_income_missed))

Доля пропущенных значений в столбце 'days_employed': 10.1%
Доля пропущенных значений в столбце 'total_income': 10.1%


Доля пропусков для стоблцов `days_employed` и `total_income` совпадает. Возможно пропуски вызваны одной причиной. Проверим это предположение:

In [3]:
#Определим количество строк, в которых пропущенны записи только в одном столбце:
len(data[(data['days_employed'].isna()) != (data['total_income'].isna())])

0

Предположение подтвердилось - пропуски в столбцах `days_employed` и `total_income` наблюдаются только одновременно. <br>
Подробнее ознакомися со строками, в которых есть пропуски

In [4]:
data[(data['days_employed'].isna())].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


На первый взгляд записи содержащие пропуски имеют мало общего между собой. Проверим, отличаются ли распределение значений в других столбцах в зависимости от пропусков в `days_employed`:

In [5]:
display(data[(data['days_employed'].isna())].describe())
display(data[(data['days_employed'].notna())].describe())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,1.935100e+04
mean,0.537388,63046.497661,43.255336,0.819079,0.972249,0.081184,1.674223e+05
std,1.371408,140827.311974,12.579170,0.550104,1.420596,0.273125,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Видно что описательные статистики не зависят от пропусков в столбцах `days_employed` и `total_income`.<br>
Поскольку наличие или отсутствие пропусков не связано с параметрами клиентов, лучшим решением будет заполнить пропуски медианным значением.<br>
Заполним пропуски:

In [6]:
days_employed_median = data['days_employed'].median()
total_income_median = data['total_income'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['total_income'] = data['total_income'].fillna(total_income_median)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски успешно заполнены.<br>
Вероятнее всего причина возникновения пропусков является технической поскольку наличие или отсутствие пропусков не связано с параметрами клиентов. Одновременные пропуски в столбцах `days_employed` и `total_income` могут объясняться особенностями формы, в которую вносятся данные. Например, клиент не указывает место работы и поэтому не может указать стаж работы и доход.<br>
Можно предположить и другие причины, однако проще и правильнее было бы узнать об особенностях сбора этих данных напрямую.

### Проверка данных на аномалии и исправление аномальных данных

Количественные данные охарактеризуем при помощи описательных статистик:

In [7]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Поскольку <i>количество детей</i> и <i>трудовой стаж</i> не могут быть отрицательными, наличие таких значений в столбцах `children` и `days_employed` очевидно ошибочно.<br>

#### Проверка и исправление аномалий в столбце `days_employed`
Отрицательными являются более 75% значений в столбце `days_employed`, максимальное значение трудового стажа также вызывает подозрение:

In [8]:
#Доля записей с положительным значением стажа:
print('Доля записей с положительным стажем:', data[data['days_employed'] > 0]['days_employed'].count() / len(data))

#Перевод максимального стажа в годы
max_days = data[data['days_employed'] > 0]['days_employed'].max()/365
print("Максимальный стаж, лет: ", max_days)

#Перевод минимального положительного стажа в годы
min_plus_days = data[data['days_employed'] > 0]['days_employed'].min()/365
print("Минимальный положительный стаж, лет: ", min_plus_days)

#Строки с максимальным значением стажа:
data[data['days_employed'] > 0].head()

Доля записей с положительным стажем: 0.16004645760743322
Максимальный стаж, лет:  1100.6997273296713
Минимальный положительный стаж, лет:  900.6266317932007


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


Все положительные значения стажа укладываются в диапазон 900-1100 лет и очевидно ошибочны.<br>
Прочие параметры в этой строке выглядят вполне правдоподобно. Можно заметить, что преобладающей категорией по доходам является "пенсионер", проверим, так ли это, возможно некорректное указание трудового стажа связано именно с типом доходов:

In [9]:
data[data['days_employed'] > 0].groupby('income_type').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
безработный,2,2,2,2,2,2,2,2,2,2,2
пенсионер,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443,3443


Подавляющее большинство записей с положительным стажем относится к людям пенсионным типом занятости. Сравним с данными для всего набора:

In [10]:
data.pivot_table(index='income_type', aggfunc=('count', 'median'))['days_employed']

,count,median
income_type,,
безработный,2,366413.652744
в декрете,1,-3296.759962
госслужащий,1459,-2385.358043
компаньон,5085,-1311.128244
пенсионер,3856,360505.668544
предприниматель,2,-862.108806
сотрудник,11119,-1360.363902
студент,1,-578.751554


<p>Таким образом положительный стаж указан у всех безработных (2 строки) и значительной части пенсионеров.</p>
<p>Для установления причин аномалий в столбце стажа следовало бы обратиться в отдел, отвечающий за подготовку представленных данных, или по-крайней мере выяснить как происходит сбор и ввод представленных данных.</p>
<p>Рассчет продолжительно стажа в днях вряд ли осуществляется клиентами лично, более вероятен расчет по введенным клиентом датам начала и окончания трудовой деятельности в конкретных организациях. </p>
<p>Таким образом отрицательные значения могли быть получены при неправильном порядка вычитания дат (из даты начала работы вычитается дата окончания) и правильным будет их преобразования в положительные.</p>
<p>Пенсионеры и безработные могли вообще не указывать даты начала или окончания трудового стажа. Объяснить получение стажа именно в 900-1100 лет крайне сложно, лучшим решением было бы обратиться к составителям представленного набора данных.</p>
<p>В задачи исследования не входит анализ влияния трудового стажа, поэтому аномалии в данных по трудовому стажу не являются критическими. Можно предложить следующие решения:</p>
<ul>
    <li>Оставить данные в столбце `days_eployed` как есть и проигнорировать их;</li>
    <li>Установить для клиентов с типом занятости "пенсионер" значение трудового стажа в соответствии с минимальными требованиями законодательства РФ, в 2021 г.этот показатель составлял 12 лет ~ 4380 дней;</li>
    <li>Количество клиентов с типом занятости "безработный" слишком мало (2 из 21525) чтобы по ним можно было строить какие-либо статистические выводы, а также чтобы повлиять на результат анализа влияния остальных параметров. Строки с таким типом занятости могут быть исключены.</li>
</ul>
<p>Пункты 2 и 3 позволят сохранить набольшее количество данных и привести их к максимально правдоподобному виду.</p>

In [11]:
#Изменение стажа у "пенсионеров" с положительным стажем:
data.loc[(data['income_type'] == 'пенсионер') & (data['days_employed'] > 0), 'days_employed'] = 12*365

#Удаление строк с типом занятости "безработный":
data = data.drop(data.index[data['income_type'] == 'безработный'])

#Перевод отрицательного стажа в положительный:
data['days_employed'] = data['days_employed'].abs()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,4380.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Проверим не осталось ли аномальных значений трудового стажа, например превышающих возраст:

In [12]:
print("Количество строк, которых стаж больше возраста:", len(data[data['days_employed']/365 > data['dob_years']]))
print("Распределение аномальных записей по типам занятости:")
display(data[data['days_employed']/365 > data['dob_years']].groupby('income_type')['dob_years'].count())

Количество строк, которых стаж больше возраста: 101
Распределение аномальных записей по типам занятости:


income_type
госслужащий     6
компаньон      20
пенсионер      20
сотрудник      55
Name: dob_years, dtype: int64

Осталось 100 строк с аномальным значением возраста. Возможно три решения:
<ul>
    <li>Оставить значения "как есть", поскольку возраст не учитывается при решении задач исследования;</li>
    <li>Исключить строки. 100 записей составляют около 0.46% от всех данных и их исключение не приведет к существенному изменению конечных результатов;</li>
    <li>Заменить нулевые значения минимальным пенсионным возрастом для пенсионеров и медианным возрастом для остальных групп.</li>
    </ul>
В рамках текущего исследования все три решения практически равноценны. Предпочтительным будет первое как наиболее простое и сохраняющее исходные данные.<br>

#### Проверка и исправление аномалий в столбце `children`<br>
<p>Вызывает подозрение максимальное (20) и минимально количетсво детей (-1)</p>
<p>Определим количество строк, содержащих различные значения:</p>

In [13]:
data.groupby('children').count()['debt']

children
-1        47
 0     14148
 1      4817
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64

Аномальные значения могли возникнуть как опечатки при вводе данных ("-1" вместо "1" и "20" вместо "2"), как артефакты при выгрузке данных ("-1" теоретически могло указывать на отсутствие значения), а также умышленно (к примеру форма ввода позволяет выбрать число детей от 1 до 20 и пользователь указал максимально возможное).<br>
Чтобы установить истинную причину аномалий необходимо связаться с отделами ответственными за формирование и выгрузку представленных данных.<br>
Поскольку минимальное и максимальное значение количества детей встречаются относительно нечасто (~ 0,6 %), без дополнительной информации будет наболее правильно пропустить строки с аномальными значениями.<br>

In [14]:
anomalous_children = (data['children'] == -1) | (data['children'] == 20)
data = data.drop(data.index[anomalous_children])
children_debt_data = data.pivot_table(index='children', aggfunc=('mean', 'count','std'))['debt']

children_debt_data.columns = ['Число записей', 'Средняя доля просрочек', 'Стандартное отклонение']
children_debt_data

,Число записей,Средняя доля просрочек,Стандартное отклонение
children,,,
0,14148.0,0.075134,0.263617
1,4817.0,0.091966,0.289008
2,2055.0,0.094404,0.292461
3,330.0,0.081818,0.274504
4,41.0,0.097561,0.300406
5,9.0,0.000000,0.000000


Как видно из полученной таблицы, средняя доля просроченных платежей по кредиту для клиентов без детей несколько меньше чем у клиентов, имеющих одного или более детей.<br>
Говорить о значимых различиях в доле просрочек среди клиентов с детьми нельзя. Отсутствие просрочек у клиентов с 5 детьми, может быть связано с малым количеством таких клиентов.


### Изменение типов данных.

Преобразуем доход и трудовой стаж в днях в целочисленные тип данных:

In [15]:
data['days_employed'] = data['days_employed'].astype('uint16')
data['total_income'] = data['total_income'].astype('uint32')

Столбцы с небольшими значениями (число детей, возраст), индикаторы и идентификаторы преобразуем в тип `uint8`:

In [16]:
#Столбцы с малыми положительными значениями (индикаторы и т.д.):
small_positive_cols = ['children', 'dob_years', 'education_id', 'family_status_id', 'debt']
for col in small_positive_cols:
    data[col] = data[col].astype('uint8')
data.reset_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21400 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21400 non-null  uint8 
 1   days_employed     21400 non-null  uint16
 2   dob_years         21400 non-null  uint8 
 3   education         21400 non-null  object
 4   education_id      21400 non-null  uint8 
 5   family_status     21400 non-null  object
 6   family_status_id  21400 non-null  uint8 
 7   gender            21400 non-null  object
 8   income_type       21400 non-null  object
 9   debt              21400 non-null  uint8 
 10  total_income      21400 non-null  uint32
 11  purpose           21400 non-null  object
dtypes: object(5), uint16(1), uint32(1), uint8(5)
memory usage: 1.7+ MB


Удалось сократить использование памяти. В данном случае это не является критичным, однако может быть полезно при увеличении числа записей в наборе данных.

### Удаление дубликатов.

Удалим явные дубликаты:

In [17]:
print("Количество записей до удаления явных дубликатов:", len(data))
data = data.drop_duplicates().reset_index(drop=True)
print("Количество записей после удаления явных дубликатов:", len(data))

Количество записей до удаления явных дубликатов: 21400
Количество записей после удаления явных дубликатов: 21346


Возможно наличие неявных дубликатов. Рассмотрим возможные уникальные значения в столбцах `education`, `family_status`, `purpose`

In [18]:
print("Варианты образования:")
print(data['education'].unique())
print()
print("Варианты семейного положения:")
print(data['family_status'].unique())
print()
print("Варианты целей кредитования:")
print(data['purpose'].unique())

Варианты образования:
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Варианты семейного положения:
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Варианты целей кредитования:
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобил

Присутствуют различные написания ступеней образования и различные но близкие формулировки целей кредита.<br>
Цели кредита будут категоризированы далее, в рамках задачи исследования.<br>
Варианты ступеней образования могут быть унифицированы переводом в нижним регистр:

In [19]:
#Переведем в нижний регистр:
data['education'] = data['education'].str.lower()

#Проверим наличие дубликатов:
print("Количество записей до удаления явных дубликатов:", len(data))
data = data.drop_duplicates().reset_index(drop=True)
print("Количество записей после удаления явных дубликатов:", len(data))

Количество записей до удаления явных дубликатов: 21346
Количество записей после удаления явных дубликатов: 21329


### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Подготовка датафреймов-словарей для уровней образования и семейного положения:

In [20]:
#Датафрейм словарь для уровней образования:
education_ids = data[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
print("Словарь уровней образования")
display(education_ids)
print()

#Датафрейм словарь для семейного положения:
family_status_ids = data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
print("Словарь семейного положения")
display(family_status_ids)

Словарь уровней образования


,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4



Словарь семейного положения


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Декомпозиция исходного датафрейма:

In [21]:
data = data.drop(['family_status', 'education'], axis=1)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,4380,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


 Проверим влияние образования на долю просроченных кредитов:

In [22]:
data.pivot_table(index='education_id',
                 aggfunc=('mean', 'count','std'))['debt'].merge(education_ids, on='education_id')

,education_id,count,mean,std,education
0,0,5226.0,0.053196,0.224445,высшее
1,1,15074.0,0.089824,0.285938,среднее
2,2,741.0,0.091768,0.288893,неоконченное высшее
3,3,282.0,0.109929,0.313357,начальное
4,4,6.0,0.000000,0.000000,ученая степень


<p>Заметен большой разброс доли просроченных кредитов в зависимости от уровня образования, наибольший риск наблюдается в группе с начальным образованием, группа со средним и неоконченным высшим образованием имеет близкий уровень риска, наименьший риск наблюдается в группе с высшим образованием. Нулевая доля просроченных кредитов в группе с учеными степенями скорее всего объясняется слишком малым количеством наблюдений (6).</p>
<p>Такое распределение риска в зависимости от образования может быть объяснено тем, что более квалифицированные клиенты имеют более постоянные доходы.</p>

In [23]:
data.pivot_table(index='family_status_id',
                 aggfunc=('mean', 'count','std'))['debt'].merge(family_status_ids, on='family_status_id')

,family_status_id,count,mean,std,family_status
0,0,12260.0,0.075530,0.264256,женат / замужем
1,1,4133.0,0.093153,0.290681,гражданский брак
2,2,951.0,0.066246,0.248843,вдовец / вдова
3,3,1189.0,0.070648,0.256343,в разводе
4,4,2796.0,0.097639,0.296880,Не женат / не замужем


<p>Наименьший риск наблюдается в группах лиц, которые либо находятся в официальном браке ("женат/замужем"), или были в нем ранее ("вдовец/вдова", "в разводе"), причем уровень риска в данных группах практически не различается.</p>
<p>Наибольший кредитный риск наблюдается в группах "гражданский брак" и "Не женат / не замужем" и также мало различается между этими группами.</p>
<p>Причиной низкой доли просроченных кредитов в группе находящихся в официальном браке является возможность поддержки клиента, взявшего кредит, его супругом в случае возникновения финансовых трудностей.</p>
<p>Причины низкого кредитного риска для разведенных и вдовых клиентов не столь ясна. Одной из причин может быть различие в возрасте клиентов. Проверим эту гипотезу.</p>

In [24]:
data.pivot_table(index='family_status_id',
                 aggfunc=('mean', 'count','std'))['dob_years'].merge(family_status_ids, on='family_status_id')

,family_status_id,count,mean,std,family_status
0,0,12260.0,43.559543,11.936701,женат / замужем
1,1,4133.0,42.067022,12.352777,гражданский брак
2,2,951.0,56.501577,9.583964,вдовец / вдова
3,3,1189.0,45.561817,11.800556,в разводе
4,4,2796.0,38.368026,13.322453,Не женат / не замужем


Действительно, возраст разведенных и вдовых клиентов больше возраста клиентов в остальных группах. Проверим влияет ли возраст на вероятность просрочки по кредиту.

In [25]:
data.pivot_table(index='debt', aggfunc=('mean', 'count','std'))['dob_years']

,count,mean,std
debt,,,
0,19598.0,43.537912,12.589283
1,1731.0,40.340843,12.039120


Возраст клиентов, не допускающих просрочек по кредиту, значимо выше возраста клиентов с просрочками. Это <i>может быть</i> причиной низкого кредитного риска в группах клиентов, ранее находившихся в браке.

### Категоризация дохода

In [26]:
#Функци для категоризации дохода:
def income_categorizer(total_income):
    """Функция категоризирует клиентов в зависимости от суммарного дохода
    """
    try:
        if total_income < 30001:
            return "E"
        elif total_income < 50001:
            return "D"
        elif total_income < 200001:
            return "C"
        elif total_income < 1000001:
            return "B"
        else:
            return "A"
    except:
        print("Проверьте введённые данные:", total_income)
        return np.NaN

data['total_income_category'] = data['total_income'].apply(income_categorizer)
income_debt_data = data.pivot_table(index='total_income_category', aggfunc=('mean', 'count','std'))['debt']

income_debt_data.columns = ['Число записей', 'Доля просрочек', 'Стандартное отклонение']
income_debt_data

,Число записей,Доля просрочек,Стандартное отклонение
total_income_category,,,
A,25.0,0.080000,0.276887
B,5012.0,0.070630,0.256232
C,15921.0,0.084919,0.278770
D,349.0,0.060172,0.238147
E,22.0,0.090909,0.294245


По полученным результатам можно сделать вывод о небольшом различии доли просрочек у клиентов из групп доходов <b>B</b> и <b>C</b>, увеличение дохода при этом приводит к снижению доли просроченных кредитов.<br>
Относительно малое количество клиентов в других категориях не позволяет сделать выводы о различии доли просрочек.<br>
Можно попробовать исправить это изменив категоризацию доходов:

In [27]:
#Изменим функцию категоризатор, сделав разбивку по квантилям дохода
def income_categorizer_quant(total_income, quantiles):
    """Функция категоризирует клиентов в зависимости от суммарного дохода
       Категоризация прозводится по квантилям доходов.
    """
    categories = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    try:
        total_income = int(total_income)
    except:
        print("Проверьте введённые данные:", total_income)
        return np.NaN
    
    for i in range(len(quantiles)):
        if total_income > quantiles[i]:
            return categories[i]
    return categories[len(quantiles)]
    
#Определение квантилей распределения доходов:
quantiles = [0.8, 0.6, 0.4, 0.2]
quantiles = list(data['total_income'].quantile(quantiles))
print("Пороговые значения доходов:", quantiles)

#Категоризация доходов:
data['total_income_category'] = data['total_income'].apply(income_categorizer_quant, quantiles=(quantiles))
income_debt_data = data.pivot_table(index='total_income_category', aggfunc=('mean', 'count','std'))['debt']

income_debt_data.columns = ['Число записей', 'Доля просрочек', 'Стандартное отклонение']
income_debt_data

Пороговые значения доходов: [214587.20000000004, 156446.8, 135418.6, 98518.8]


,Число записей,Доля просрочек,Стандартное отклонение
total_income_category,,,
A,4266.0,0.070089,0.255327
B,4266.0,0.086029,0.280440
C,4265.0,0.086753,0.281505
D,4266.0,0.082513,0.275177
E,4266.0,0.080403,0.271948


Исходя из полученных данных можно сделать вывод о том, что уровень доходов не влияет на вероятность просрочки платежа по кредиту.<br>
Даже низкое значение просрочек для клиентов с наибольшими доходами недостаточно сильно отличается от остальных значений.

### Категоризация целей кредита

In [28]:
# Составим словарь катергорий
purpose_categories = {
    #операции с автомобилем включают различные варианты слова "автомобиль"
                      'операции с автомобилем' : ['автомобил'],
    #операции с недвижимостью включают различные варианты слов "недвижимость" и "жилье"
                      'операции с недвижимостью' : ['жиль', 'недвижимост', ],
    #проведение свадьбы включает различные варианты слова "свадьба"
                      'проведение свадьбы' : ['свадьб'],
    #получение образования включает различные варианты слова "образование"
                      'получение образования' : ['образовани']}

# Функция для категоризации целей кредитования
def purpose_categorizer(purpose, categories):
    """Функция категоризирует клиентов в зависимости от целей кредитования
       Категоризация прозводится по по категориям, указанным в словаре categories.
    """
    for category, words in categories.items():
        for word in words:
            if word in purpose:
                return category
    return None

#Категоризация целей кредитования:
data['purpose_category'] = data['purpose'].apply(purpose_categorizer, categories=(purpose_categories))

#Проверка категоризации целей кредитования:
print("Строк с неустановленной категорией:", data['purpose_category'].isna().sum())

#Создание сводной таблицы по целям кредитования
purpose_debt_data = data.pivot_table(index='purpose_category', aggfunc=('mean', 'count','std'))['debt']
purpose_debt_data.columns = ['Число записей', 'Доля просрочек', 'Стандартное отклонение']
purpose_debt_data

Строк с неустановленной категорией: 0


,Число записей,Доля просрочек,Стандартное отклонение
purpose_category,,,
операции с автомобилем,4279.0,0.093480,0.291138
операции с недвижимостью,10749.0,0.072472,0.259280
получение образования,3988.0,0.092528,0.289806
проведение свадьбы,2313.0,0.079118,0.269981


Категории "операции с автомобилем" и "получение образования" характеризуются близкой и относительно большой долей просрочек по кредиту.<br>
Объяснением такой тенденции могли бы быть различия в доходах, возрасте клиентов, а также "серьезности их намерений".<br>
Проверим как меняется возраст и доход в зависимости от целей кредитования:

In [29]:
data.pivot_table(index='purpose_category', aggfunc=('median', 'mean'))[['dob_years', 'total_income']]

dob_years          total_income          
                               mean median           mean    median
purpose_category                                                   
операции с автомобилем    43.503622   43.0  165148.106801  145017.0
операции с недвижимостью  43.169876   42.0  166643.770118  145017.0
получение образования     43.415496   42.0  162435.556921  145017.0
проведение свадьбы        43.130134   42.0  163838.010376  145017.0

<p>Как видно из таблицы, средние значения возраста и дохода для всех категорий близки, а медианные полностью совпадают (за исключение возраста при покупке автомобиля).</p> 
<p>Таким образом наиболее вероятной причиной различной доля просрочек является "серьезности намерений" клиентов. Возможно клиенты более взвешено подходят к получению кредита в случае покупки недвижимости и проведения свадьбы. Приобретаемая недвижимость может являться единственным жильем клиента, что будет мотивировать его более дисциплинировано выплачивать кредит.</p>
<p>Кроме того, покупка недвижимости зачастую производится семейными парами, а в свадьба обязательно участвуют как минимум двое молодоженов и, возможно, их родители. Таким образом в обоих случаях с большой вероятностью будут участвовать другие лица, которые могут помочь с выплатой кредита в случае финансовых затруднений клиента</p>

## Ответы на вопросы

### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

### Вывод 1:

<p>Средняя доля просроченных платежей по кредиту для клиентов без детей несколько меньше чем у клиентов, имеющих одного или более детей.</p>
<p>Говорить о значимых различиях в доле просрочек среди клиентов с детьми нельзя. Отсутствие просрочек у клиентов с 5 детьми, может быть связано с малым количеством таких клиентов.</p>
<p>Это может быть связано с возникновением непредвиденных расходов родителей. При этом количество таких непредвиденных расходов значимо не зависит от числа детей в семье.</p>

### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

### Вывод 2:

<p>В зависимости от семейного положения клиентов можно разделить на группу низкого и высокого кредитного риска.</p>
<p>Группа низкого риска включает клиентов, находящихся в официальном браке ("женат/замужем"), или находившихся в нем ранее ("вдовец/вдова", "в разводе"), причем уровень риска в данных группах практически не различается.</p>
<p>Группа высокго риска включает клиентов, не состоящих и не состоявших в официальном браке - "гражданский брак" и "не женат / не замужем"</p>
<p>Причиной низкой доли просроченных кредитов в группе находящихся в официальном браке является возможность поддержки клиента, взявшего кредит, его супругом в случае возникновения финансовых трудностей.</p>
<p>Причиной низкого кредитного риска в группах клиентов, ранее находившихся в браке <i>может быть</i> возраст клиентов. Анализ показывает, что в группе клиентов, не допускающих просрочек по кредиту, возраст значимо выше возраста клиентов с просрочками. Данный вывод является предварительным и требует дополнительной проверки.</p>

### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

### Вывод 3:

<p>Уровень доходов не влияет на вероятность просрочки платежа по кредиту.</p>
<p>Группа клиентов с наибольшими доходами имеет несколько меньшую долю просрочки по кредиту, однако различие находится не является значимым.</p>

### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

### Вывод 4:

<p>Категории "операции с автомобилем" и "получение образования" характеризуются близкой и относительно большой долей просрочек по кредиту.</p>
<p>В случае "операций с недвижимостью" и "проведения свадьбы" доля просрочек ниже. Представленные данные не показывают свзи целей кредита с другими параметрами.<p>
<p>Возможно клиенты более взвешено подходят к получению кредита в случае покупки недвижимости и проведения свадьбы. Приобретаемая недвижимость может являться единственным жильем клиента, что будет мотивировать его более дисциплинировано выплачивать кредит.</p>
<p>Кроме того, в покупе недвижимости и проведении свадьбы помимо лица, получаеющего кредит, участвуют и другие люди (супруг/супруга, родители молодоженов и другие родственники), которые могут помочь с выплатой кредита в случае финансовых затруднений клиента</p>

## Общий вывод:

Проведена предварительная обработка данных (исключение пропусков, удаление дубликатов). Клиенты категоризированы по цели кредитования и уровню дохода\

Проверено влияние следующих параметров клиентов на вероятность возврата кредита в срок:

* количество детей;
* семейное положение;
* уровень дохода;
* цель кредита.

Наиболее сильные различия в доле просроченных кредитов наблюдаются для групп с различными *целями кредитования* и *семейным положением*.

**По семейному положению:**
- высокий кредитный риск наблюдается в группах лиц, *не состоящих и не состоявших в официальном браке* ("гражданский брак", "не женатые / не замужние");
- низкий риск наблюдается для лиц либо *состоящих или ранее состоявших в браке* ("женат / замужем", "вдовец / вдова", "в разводе").

- высокий кредитный риск в группах "операции с автомобилем" и "получение образования" и меньший риск в группах "операции с недвижимостью" и "проведение свадьбы".

**По целям кредитования:**
- низкий риск характерен для целей "операции с недвижимостью" и "проведение свадьбы";
- высокий риск характерен для целей "операции с автомобилем" и "получение образования".

**По количеству детей:**\
Средняя доля просроченных платежей по кредиту для клиентов без детей несколько меньше чем у клиентов, имеющих одного или более детей. Доля просроченных кредитов не связана с количеством детей (1-5) при их наличии.

Влияния **уроня доходов** на долю просроченных кредитов не обнаружено.
